# Projet Final sur IMdb

Récupération des données sur les films sorties depuis 2017. 
Les données sont :
    - Le nom du film
    - L'année de sortie
    - le genre
    - Le ratio 
    - le nombre de votes 
    - le métascore 

Le but étant de prédire quels films ont le plus du succès par rapport au genre. 

### importation des modules

In [3]:
import urllib
import bs4
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output
from time import time

In [4]:
import json
import pandas as pd
import requests

In [5]:
import pymysql

## 1ère étape: Récupération des données 

In [6]:
#Page 

url_movie = 'http://www.imdb.com/search/title?title_type=feature&release_date='

hdrs = {'User-Agent': 'Brave / 5.0 (X11 Linux x86_64) AppleWebKit / 537.36 (KHTML, like Gecko) Chrome / 52.0.2743.116 Safari / 537.36'}    

request = requests.get(url_movie, headers = hdrs)
request_text = request.text
page = bs4.BeautifulSoup(request_text)

### récupération un à un des éléments

In [7]:
# récupartion du titre de la recherche
page.find('h1', {'class' : "header"}).text

'Feature Film\n(Sorted by Popularity Ascending) '

In [6]:
#récupération du texte sur toute la page de recherche 
page.find('div', {'class' : "lister-list"}).text

g but selfish young woman meddles in the love lives of her friends.\n\n    Director:\nAutumn de Wilde\n| \n    Stars:\nAnya Taylor-Joy, \nJohnny Flynn, \nBill Nighy, \nMia Goth\n\n\nVotes:\n5,097\n\n\n\n\n\n\n\n\n \n \n\n\n16.\nBaaghi 3\n(2020)\n\n\n143 min\n|\n\nAction, Thriller            \n\n\n\n\n1.9\n\n\n\n\nRate this\n\n\n\n\n\n\n\xa0\n\xa0\n\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n\n1.9/10\nX\n\xa0\n\n\n\n\n    A man embarks on a bloody rampage to save his kidnapped brother.\n\n    Director:\nAhmed Khan\n| \n    Stars:\nTiger Shroff, \nRiteish Deshmukh, \nShraddha Kapoor, \nAnkita Lokhande\n\n\nVotes:\n3,087\n\n\n\n\n\n\n\n\n \n \n\n\n17.\nJungle Cruise\n(2020)\n\n\n\nAction, Adventure, Comedy            \n|\nPost-production\n\n\n    Based on Disneyland\'s theme park ride where a small riverboat takes a group of travelers through a jungle filled with dangerous animals and reptiles but with a supernatural element.\n\n    Director:\nJaume Collet-Serra\n| \n    Stars:\nEmily Blunt, \nDwayn

In [7]:
#référence du film avce le titre
page.h3.a

<a href="/title/tt8629748/?ref_=adv_li_tt">Spenser Confidential</a>

In [8]:
#récupération du titre du film
page.h3.a.text

'Spenser Confidential'

In [9]:
#récupération de l'année 
page.find('span', {'class' : "lister-item-year text-muted unbold"}).text

'(2020)'

In [ ]:
#récuparation des genres du film
page.find('span', {'class' : "genre"}).text

In [11]:
#Récupération des notes étoiles 
page.find('div', {'class' : "inline-block ratings-imdb-rating"}).text

'\n\n6.2\n'

In [12]:
#Récupération des métascores 
page.find('div', {'class' : "inline-block ratings-metascore"}).text

'\n49        \n        Metascore\n            '

In [13]:
#Récupération des votes
page.find('span', {'name' : "nv"}).text

'24,517'

### Boucle pour récupérer autotmatiquement toutes les donées du films

In [14]:
#Prendre le nombre de page par année et le nombre d'années à scraper

pages = [str(i) for i in range(1,50)]
years_url = [str(i) for i in range(2017,2019)]	
headers = {"Accept-Language": "en-US, en;q=0.5"}

In [15]:
# Création d'une liste avec les données à récupérer 
names = [] 
years = []
imdb_ratings = []
metascores = []
votes = []
genres = []

# Le nombre de pages par année et le nombre d'année à scraper
pages = [str(i) for i in range(1,50)]
years_url = [str(i) for i in range(2017,2019)]	
headers = {"Accept-Language": "en-US, en;q=0.5"}

# Liste des données à récupérer
names = [] 
years = []
imdb_ratings = []
metascores = []
votes = []
genres = []

# Décompte des pages récupérer
start_time = time()
requests_number = 0

# Pour toutes les années dans l'interval
for year_url in years_url:
    
    # Pour toutes les années de l'interval
    for page in pages:
        
        # Faire la requete via le IMDb
        response = get('http://www.imdb.com/search/title?title_type=feature&release_date=' + year_url + 
        '&sort=num_votes,desc&start=' + page + '&ref_=adv_nxt', headers = headers)
        
        # Pause the loop
        sleep(randint(8,15))

        # Voir le nombre de films recupérés avec le temps
        requests_number += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests_number, requests_number/elapsed_time))
        clear_output(wait = True)
        
        # Avertissement quand il y un problème 
        if response.status_code != 1000:
            warn('Request: {}; Status code: {}'.format(requests_number, response.status_code))
        
        # Brisez la boucle si le nombre de pages est supérieur à 80 pages
        if requests_number > 80:
            warn('Number of requests was greater than expected.')  
            break 

        # Analyser le contenu de la demande avec BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Sélectionnez tous les 50 conteneurs de film sur une seule page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # Pour ces 50 films 
        for container in mv_containers:

            # Inclure seulement si le film est une "Metacore" 
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape le nom
                name = container.h3.a.text
                names.append(name)

                # Scrape l'année 
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                 # Scrape le ratio IMDB
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape le Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape le nombre de votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
                
                # Scrape le genre
                genre = container.find('span', class_ = 'genre').text
                genres.append(genre.strip())

Request:81; Frequency: 0.07465865700280956 requests/s


In [16]:
container.find('div', class_ = 'ratings-metascore')

<div class="inline-block ratings-metascore">
<span class="metascore unfavorable">31        </span>
        Metascore
            </div>

In [17]:
df_movie = pd.DataFrame({'movie':names,
'years':years,'imdb':imdb_ratings,'genre':genres,'votes':votes,
'metascore':metascores})

print(df_movie.info())
df_movie.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 6 columns):
movie        3921 non-null object
years        3921 non-null object
imdb         3921 non-null float64
genre        3921 non-null object
votes        3921 non-null int64
metascore    3921 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 183.9+ KB
None


,movie,years,imdb,genre,votes,metascore
0,Logan,(2017),8.1,"Action, Drama, Sci-Fi",603294,77
1,Thor: Ragnarok,(2017),7.9,"Action, Adventure, Comedy",536979,74
2,Star Wars: Episode VIII - The Last Jedi,(2017),7.0,"Action, Adventure, Fantasy",527314,85
3,Guardians of the Galaxy Vol. 2,(2017),7.6,"Action, Adventure, Comedy",527257,67
4,Wonder Woman,(2017),7.4,"Action, Adventure, Fantasy",519811,76
5,Dunkirk,(2017),7.9,"Action, Drama, History",504922,94
6,Spider-Man: Homecoming,(2017),7.4,"Action, Adventure, Sci-Fi",474162,73
7,Get Out,(I) (2017),7.7,"Horror, Mystery, Thriller",447628,84
8,It,(I) (2017),7.3,Horror,424133,69
9,Blade Runner 2049,(2017),8.0,"Action, Drama, Mystery",417353,81


## Extraire le dataframe en CSV 

In [20]:
df_movie.to_csv(r'C:\Users\utilisateur\Desktop\Simplon DATA IA\PROJET FINAL\movie.csv', index = False)

## 2ème étape: Stockage des données dans une base SQL

In [ ]:
fichier : "BDD movie.ipynb"

In [2]:
fichier : "bdd.mwb", "movie.sql"

NameError: name 'bdd' is not defined

## 3ème étape: Analyse des données et visualisations


In [ ]:
fichier : "visualisation donnée.ipynb"

## 4ème étape: Mise en place d’un modèle de ML

In [ ]:
fichier : "visualisation donnée.ipynb" 